In [1]:
import datetime
import numpy as np
from osgeo import gdal

In [29]:
data_in = gdal.Open(r"convert_2015_input.dat")
im_width = data_in.RasterXSize
im_height = data_in.RasterYSize
im_band = data_in.RasterCount
im_geotrans = data_in.GetGeoTransform()
im_proj = data_in.GetProjection()

In [30]:
data_aux = gdal.Open(r"ml_2015_input.dat")
aux = data_aux.ReadAsArray(0, 0, im_width, im_height)
veg = aux[6, :, :]
clay = aux[9, :, :]
veg=veg.reshape(-1,1)
clay=clay.reshape(-1,1)

In [31]:
zheight = 1000  # height where wind speed was measured, cm
z0 = 7.9  # roughness length, cm
A = 0.026
p = 0.00129   # g/cm3
g = 980  # cm/s2

u = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])*100  # windspeed
ustar_hist = np.array([wind1, wind2, wind3, wind4, wind5,
                       wind6, wind7, wind8, wind9, wind10])  # frequency
ustar = (u*0.4)/np.log10(zheight/z0)
ustar_hist = ustar_hist/np.sum(ustar_hist)

In [32]:
data_read = data_in.ReadAsArray(0, 0, im_width, im_height)
omega_ns1 = data_read[0, :, :]
omega_ns2 = data_read[1, :, :]
omega_ns3 = data_read[2, :, :]
omega_ns4 = data_read[3, :, :]
omega_ns5 = data_read[4, :, :]
omega_ns6 = data_read[5, :, :]
omega_ns7 = data_read[6, :, :]

In [33]:
# build wind pdf
wind1 = 0
wind2 = 0.1
wind3 = 0.4
wind4 = 0.192
wind5 = 0.133
wind6 = 0.083
wind7 = 0.042
wind8 = 0.05
wind9 = 0
wind10 = 0

In [35]:
L_omega1 = omega_ns1 / 50
L_omega2 = omega_ns2 / 50
L_omega3 = omega_ns3 / 50
L_omega4 = omega_ns4 / 50
L_omega5 = omega_ns5 / 50
L_omega6 = omega_ns6 / 50
L_omega7 = omega_ns7 / 50

mask_sum=np.sum(data_in,axis=0)
mask = np.where(mask_sum > 0)
veg_mask=veg[mask]
clay_mask=clay[mask]
L_omega1_mask = L_omega1.reshape(-1,1)[mask]
L_omega2_mask = L_omega2.reshape(-1,1)[mask]
L_omega3_mask = L_omega3.reshape(-1,1)[mask]
L_omega4_mask = L_omega4.reshape(-1,1)[mask]
L_omega5_mask = L_omega5.reshape(-1,1)[mask]
L_omega6_mask = L_omega6.reshape(-1,1)[mask]
L_omega7_mask = L_omega7.reshape(-1,1)[mask]

L_array = np.array([L_omega1_mask, L_omega2_mask, L_omega3_mask, L_omega4_mask,
                    L_omega5_mask, L_omega6_mask, L_omega7_mask])

In [42]:
c=2015
for z in range(7):
    temp_L = L_array[z]
    # wemo5_core
    n_windspeeds = ustar.size
    q = np.zeros((n_windspeeds))
    q_array = np.zeros((n_windspeeds))
    Ftot_veg = np.zeros(temp_L.size)
    Ftot_bare = np.zeros(temp_L.size)
    f = np.zeros(temp_L.size)
    qtot = np.zeros(temp_L.size)
    k_con = np.zeros(temp_L.size)

    for i in range(temp_L.size):
        for k in range(0, n_windspeeds):
            temp_m = 0.0000009 * ustar[k] ** 3.54
            temp_p = 0.0001 * ustar[k] ** 0.85
            q_array[k] = temp_m * np.exp((-temp_L[i] ** 1.48 / temp_p))
            q[k] = q_array[k] * ustar_hist[k]

        qtot[i] = np.sum(q)
        if qtot[i] < 0:
            qtot[i] = 0
        qtot[i] = qtot[i] * 3600 * 24
        if clay_mask[i] < 20:
            k_con[i] = np.exp(0.134 * clay_mask[i] - 6)
        else:
            k_con[i] = 0.005
        f[i] = qtot[i] * k_con[i]
        Ftot_veg[i] = f[i] * (100 - veg_mask[i]) / 100 / 30
        # Ftot_bare[i] = f[i] * bare[i] / 100

    print(datetime.datetime.now())
    # output to a image
    myarray = np.array([temp_L, qtot, Ftot_veg])
    mylist = ["temp_L", "qtot", "Ftot_veg"]
    temp = veg.reshape(im_width*im_height)
    out_image = np.zeros((len(mylist), im_height, im_width))
    for s in range(len(mylist)):
        new = np.zeros(im_width*im_height)
        new[mask] = myarray[s, :]
        out_image[s, :, :] = new.reshape(im_height, im_width)

    z1 = str(z+1)
    driver = gdal.GetDriverByName("ENVI")
    dataset = driver.Create("convert_"str(c)+"_result_"+z1+".dat",
                            im_width, im_height, len(mylist), gdal.GDT_Float32)
    dataset.SetGeoTransform(im_geotrans)
    dataset.SetProjection(im_proj)
    # dataset.GetRasterBand(1).WriteArray(out_image)
    for s, image in enumerate(out_image, 1):
        dataset.GetRasterBand(s).SetNoDataValue(0)
        dataset.GetRasterBand(s).SetDescription(mylist[s-1])
        dataset.GetRasterBand(s).WriteArray(image)

    del dataset

2023-01-27 19:02:18.849678
2023-01-27 19:05:21.335225
2023-01-27 19:08:22.303377
2023-01-27 19:11:21.649914
2023-01-27 19:14:19.039222
2023-01-27 19:17:17.572139
2023-01-27 19:20:15.618439
